In [1]:
import numpy as np
import cv2

In [6]:
cap = cv2.VideoCapture(0)

ret,frame = cap.read()

face_cascade = cv2.CascadeClassifier('../data/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

(face_x,face_y,w,h) = tuple(face_rects[0])
tracking_window = (face_x,face_y,w,h)

roi = frame[face_y:face_y+h,face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hist  =cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:

    ret, frame = cap.read()

    if ret == True:

        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # Only face needs to be in the camera at the start of capture
    
        # ret, tracking_window = cv2.meanShift(dst,tracking_window,term_crit)

        # x,y,w,h = tracking_window
        # img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)

        ret, tracking_window = cv2.CamShift(dst,tracking_window,term_crit)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True,(0,0,255),5)

        cv2.imshow('Face Tracking', img2)

        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()